In [2]:
import re
import os
import io
import pandas as pd
import numpy as np
from datetime import datetime
from tabulate import tabulate


In [3]:
#-------------------CONFIG--------------------
source_folder = r"C:\Users\Ravi Pal\my_projects\project_p767\Taxi_management_db\data\manual_operation_data"
destination_folder = r"C:\Users\Ravi Pal\my_projects\project_p767\Taxi_management_db\data\manul_files"
os.makedirs(destination_folder, exist_ok=True)

# print(os.path.exists(source_folder))  # Should be True
# print(os.path.exists(destination_folder))  # Should be True
# print(os.listdir(source_folder))      # Should list files   


In [52]:

pd.set_option('future.no_silent_downcasting', True) 
# 1. Load Data
try:
    # Step A: Read the dirty Excel file
    # header=None ensures we grab every row as raw data
    raw_df = pd.read_excel(file_path, header=None)

    # Step B: "Convert" to CSV in memory (The trick)
    csv_buffer = io.StringIO()
    raw_df.to_csv(csv_buffer, index=False, header=False)
    
    # Rewind the buffer to the beginning so we can read it
    csv_buffer.seek(0)

    # Step C: Read it back as a clean CSV
    df = pd.read_csv(csv_buffer, header=None)

    # --- NOW YOU CAN CLEAN 'df' HERE ---
    # Example cleaning:
    df = df.replace(r'^\s*$', pd.NA, regex=True) # Replace spaces
    df.dropna(how='all', inplace=True)           # Drop empty rows
    
    print(f"Processed in memory: {file} | Rows: {len(df)}")
    
    # You can now perform your logic on 'df' without saving any CSVs
    
except Exception as e:
    print(f"Error processing {file}: {e}")



target_str = "EMPLOYEE ADDRESS"
df["reporting_location"] = df[4].astype(str).apply(lambda x : x if target_str in x else "")
# Extract everything inside the first pair of double quotes
df["reporting_location"] = df["reporting_location"].str.extract(r'"([^"]*)"')
df["reporting_location"] = df["reporting_location"].ffill()
# Extract the date string (e.g., "01-12-2025")
# This assumes you have the column 'source_file' from the previous step
# 1. First, create the 'source_file' column
df['source_file'] = file  

# 2. NOW you can extract data from it
# Extract date: looks for pattern "DD-MM-YYYY"
df['date'] = df['source_file'].str.extract(r'(\d{2}-\d{2}-\d{4})')
df['reporting_date'] = pd.to_datetime(df['date'], dayfirst=True)


df = df.dropna(how="all")
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(how="all")
df = df.reset_index(drop=True)

   # 4. Rename Columns
header_mapping = {
    0: 'Trip_id', 1: 'trg_type', 2: 'Employee_ID', 3: 'Employee_Name',
    4: 'Address', 5: 'Driver_Mobile', 6: 'cab_4_digit', 7: 'pickup_time',
    8: 'shift_time', 9: 'mis_remarks', 10: 'gender', 11: 'Trip_Sheet_ID_Raw'
}
df = df.rename(columns=header_mapping)

target_str2 = "EMP ID"
if "Employee_ID" in df.columns:
    df = df[~df["Employee_ID"].astype(str).str.contains(target_str2, na=False, regex=False)]

df["Trip_id"] = df["Trip_id"].ffill()
df["Trip_No"] = "ROUTE NO : " + df["Trip_id"].astype(str)

# 1. Force convert 'date' to datetime first (Handle errors gracefully)
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')

# 2. Now you can safely use .dt accessor to get a clean string
df['DATE_STR'] = df['date'].dt.strftime('%Y-%m-%d')

# 3. Clean up SHIFT_TIME (ensure it's a string)
df['shift_time'] = df['shift_time'].astype(str).str.strip()

# 4. Combine them
df['temp_combined'] = df['DATE_STR'] + ' ' + df['shift_time']

# 5. Create the final datetime object
df['REPORTING_TIME'] = pd.to_datetime(df['temp_combined'], errors='coerce')

df['date'] = df['date'].dt.date

# Numeric Conversion
cols_to_numeric = ['Employee_ID','cab_4_digit']
df[cols_to_numeric] = df[cols_to_numeric].apply(pd.to_numeric, errors='coerce')



   # --- UPPERCASE & CLEANUP ---
df.columns = df.columns.astype(str).str.strip().str.upper()
str_cols = df.select_dtypes(include=['object']).columns
df[str_cols] = df[str_cols].apply(lambda x: x.astype(str).str.strip().str.upper())

desired_order = [
    'DATE',
    'TRIP_ID', 
    'TRIP_NO',
    'TRG_TYPE', 
    'EMPLOYEE_ID',
    'GENDER', 
    'EMPLOYEE_NAME', 
    'ADDRESS',  
    'CAB_4_DIGIT',  
    'SHIFT_TIME',
    'REPORTING_TIME', 
    'REPORTING_LOCATION',  
    'MIS_REMARKS'   
]

df = df.reindex(columns=desired_order)

Processed in memory: TRG PICKUP 08-12-2025.xlsx | Rows: 96


In [5]:
if __name__ == "__main__":
    # print(f"Scanning folder: {source_folder}")
    files_found = 0
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(('.xls', '.xlsx')):
                files_found += 1
                file_path = os.path.join(root, file)
                # clean_data(file_path, destination_folder)
    
    if files_found == 0:
        print("No Excel files found to process.")
    else:
        print("Processing complete.")
        


Processing complete.


In [50]:
# df_d = df[df['reporting_location'].notna() & (df['reporting_location'] != "")]
# print(df_d.head().to_markdown())
print(f"Shape of Data: {df.shape}")
print(df.head().to_markdown())


Shape of Data: (95, 13)
|    | DATE       |   TRIP_ID | TRIP_NO      | TRG_TYPE     |   EMPLOYEE_ID |   GENDER | EMPLOYEE_NAME          | ADDRESS                                                                                                     |   CAB_4_DIGIT | SHIFT_TIME   | REPORTING_TIME      | REPORTING_LOCATION   |   MIS_REMARKS |
|---:|:-----------|----------:|:-------------|:-------------|--------------:|---------:|:-----------------------|:------------------------------------------------------------------------------------------------------------|--------------:|:-------------|:--------------------|:---------------------|--------------:|
|  1 | 2025-12-08 |         5 | ROUTE NO : 5 | BATCH NO.326 |      80054295 |      nan | SAPNA                  | KHUSHBOO HIMANI RESIDENCE RAJPUR KHURD VILLAGE, MAIDAN GARHI, NEW DELHI, DELHI 195494                       |          7798 | 06:30:00     | 2025-12-08 06:30:00 | AITA SEC 75 GURGAON  |           nan |
|  2 | 2025-12-08 |         

In [53]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, 1 to 95
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                95 non-null     object        
 1   TRIP_ID             95 non-null     object        
 2   TRIP_NO             95 non-null     object        
 3   TRG_TYPE            95 non-null     object        
 4   EMPLOYEE_ID         95 non-null     int64         
 5   GENDER              95 non-null     object        
 6   EMPLOYEE_NAME       95 non-null     object        
 7   ADDRESS             95 non-null     object        
 8   CAB_4_DIGIT         95 non-null     int64         
 9   SHIFT_TIME          95 non-null     object        
 10  REPORTING_TIME      95 non-null     datetime64[ns]
 11  REPORTING_LOCATION  95 non-null     object        
 12  MIS_REMARKS         95 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 